In [42]:
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from typing import List, Dict, Tuple, Union
import string
import pickle
# nltk.download('punkt')
# nltk.download('stopwords')

ORIGINAL_PATH = os.path.join(os.getcwd(), os.pardir, os.pardir, 'Dataset')

In [43]:
# import os
# import random
# word_list = ['Number', 'of', 'words', 'in', 'a', 'document', 'is', 'not', 'fixed', 'and', 'can', 'vary', 'from', 'document', 'to', 'document', 'but', 'the',
# 			 'average', 'number', 'of', 'words', 'in', 'a', 'document', 'is', 'around', '200', 'to', '300', 'words', 'and', 'the', 'maximum', 'number', 'of', 'words', 'in', 'a', 'document', 'is', 'around', '1000', 'words']
# path = os.path.join(os.getcwd(), 'DS2')
# os.makedirs("DS2")
# for i2 in range(10):
# 	random.seed(i2)
# 	random_indexes = random.sample(range(0, len(word_list)), 10)
# 	filename1 = "a" + str(i2+1).zfill(2)
#
# 	with open(os.path.join(path, filename1), 'w') as f:
# 		for j2 in random_indexes:
# 			f.write(word_list[j2] + " ")

In [45]:
class BigramIndex:
    def __init__(self, path: str) -> None:
        """
        Initialize the BigramIndex object.
        :param path: The path to the collection of documents.
        """

        self.docs: int = 0
        self.index: Dict[str, Tuple[int, List[str]]] = {}
        self.PATH: str = path
        self.buildIndex()


    def buildIndex(self) -> None:
        """
        Build the bigram inverted index by processing each document in the collection.
        Process includes lower-casing, tokenizing, removing stopwords, punctuations and blank spaces.
        """

        for filename in os.listdir(self.PATH):
            self.docs += 1

            with open(os.path.join(self.PATH, filename), 'r') as f:
                content = f.read()

            # Tokenize and lower case
            tokens = word_tokenize(content.lower())
            # Remove stopwords
            tokens = [token for token in tokens if token not in stopwords.words("english")]
            # Remove punctuation
            tokens = [token.translate(str.maketrans("", "", string.punctuation)) for token in tokens]
            # Remove blank spaces
            tokens = [token for token in tokens if token.strip()]

            # Add biwords to index
            for b in range(len(tokens) - 1):
                biword = tokens[b] + " " + tokens[b+1]
                if biword not in self.index:
                    self.index[biword] = (0, [filename])
                else:
                    self.index[biword][1].append(filename)

            # Sort unique postings
            for token in self.index:
                self.index[token] = (len(self.index[token][1]), sorted(list(set(self.index[token][1]))))

        print("Finished Building Index")


    def getPostingList(self, term: str) -> Tuple[int, List[str]]:
        """
        Get the posting list for a term.
        :param term: The term to get the posting list for.
        :return: The posting list for the term as well as the frequency of the term in the collection.
        """

        return self.index[term] if term in self.index else (0, [])

    def singleWordPostingList(self, term: str) -> Tuple[int, List[str]]:
        """
        Get the posting list for a term.
        :param term: The term to get the posting list for.
        :return: The posting list for the term as well as the frequency of the term in the collection.
        """

        posting_list = []
        # Search all keys in index which contain the term and append the posting list to the list
        for key in self.index:
            if term in key:
                posting_list.append(self.index[key][1])

        # Sort unique postings
        posting_list = sorted(list(set(posting_list)))

        return len(posting_list), posting_list

    def getTotalDocs(self) -> int:
        """
        Get the total number of documents in the collection.
        :return: The total number of documents in the collection.
        """

        return self.docs


    def queryAND(self, term1: Union[str, List[str]], term2: Union[str, List[str]]) -> Tuple[List[str], int]:
        """
        Perform a boolean AND query on the bigram inverted index.
        :param term1: The first term to query or a posting list.
        :param term2: The second term to query or a posting list.
        :return: A list of document names that contain both terms and the number of comparisons made.
        """

        # Check if terms are in inverted index
        if not isinstance(term1, list) and term1 not in self.index:
            return [], 0
        if not isinstance(term2, list) and term2 not in self.index:
            return [], 0
        if isinstance(term1, list) and len(term1) == 0:
            return [], 0
        if isinstance(term2, list) and len(term2) == 0:
            return [], 0


        postings1 = self.index[term1][1] if isinstance(term1, str) else term1
        postings2 = self.index[term2][1] if isinstance(term2, str) else term2

        # Perform AND
        comparisons = 0
        result = []
        i = 0
        j = 0
        while i < len(postings1) and j < len(postings2):
            comparisons += 1
            if postings1[i] == postings2[j]:
                result.append(postings1[i])
                i += 1
                j += 1
            elif postings1[i] < postings2[j]:
                i += 1
            else:
                j += 1

        return result, comparisons


    def queryProcess(self, query: str) -> Tuple[List[str], int]:
        """
        Parse a query and perform the appropriate boolean query. Only AND query is supported.
        :param query: The query to parse, should be of the form "A B AND B C AND C D" or a similar combination.
        :return: List of document names that match the query and the number of comparisons made.
        """

        result = []
        comparisons = 0

        # Split query into AND subqueries
        and_subqueries = query.split(" ")

        if len(and_subqueries) == 1:
            return self.singleWordPostingList(and_subqueries[0])[1], 0

        bigram_subqueries = []
        for s in range(len(and_subqueries) - 1):
            bigram_subqueries.append(and_subqueries[s] + " " + and_subqueries[s+1])

        # Process each AND subquery
        for subquery in bigram_subqueries:
            # If result is empty, get posting list for first term
            if len(result) == 0:
                result = self.index[subquery][1] if subquery in self.index else []
                comparisons += 0
            # Otherwise, perform AND query
            else:
                result, subcomparisons = self.queryAND(result, subquery)
                comparisons += subcomparisons

        return result, comparisons



In [46]:
class PositionalIndex:
    def __init__(self, path: str) -> None:
        """
        Initialize the PositionalIndex object.
        :param path: The path to the collection of documents.
        """

        self.docs: int = 0
        self.index: Dict[str, List[Union[int, Dict[str, List[str]]]]] = {}
        self.PATH: str = path
        self.buildIndex()


    def buildIndex(self) -> None:
        """
        Build the bigram inverted index by processing each document in the collection.
        Process includes lower-casing, tokenizing, removing stopwords, punctuations and blank spaces.
        Index stored as {token: [frequency, {doc1: [pos1, pos2, ...], doc2: [pos1, pos2, ...], ...}]}
        """

        for filename in os.listdir(self.PATH):
            self.docs += 1

            with open(os.path.join(self.PATH, filename), 'r') as f:
                content = f.read()

            # Tokenize and lower case
            tokens = word_tokenize(content.lower())
            # Remove punctuation
            tokens = [token.translate(str.maketrans("", "", string.punctuation)) for token in tokens]
            # Remove blank spaces
            tokens = [token for token in tokens if token.strip()]


            # Add tokens to index
            for t in range(len(tokens)):
                if tokens[t] in stopwords.words("english"):
                    continue
                if tokens[t] not in self.index:
                    self.index[tokens[t]] = [1, {filename: [t+1]}]
                else:
                    if filename not in self.index[tokens[t]][1]:
                        self.index[tokens[t]][0] += 1
                        self.index[tokens[t]][1][filename] = [t+1]
                    else:
                        self.index[tokens[t]][0] += 1
                        self.index[tokens[t]][1][filename].append(t+1)


            # Sort unique postings
            for token in self.index:
                self.index[token][1] = {k: sorted(list(set(v))) for k, v in sorted(self.index[token][1].items())}


    def getPostingList(self, term: str) -> Tuple[int, Dict[str, List[str]]]:
        """
        Get the posting list for a term.
        :param term: The term to get the posting list for.
        :return: The posting list for the term as well as the frequency of the term in the collection.
        """

        if term not in self.index:
            return 0, {}

        return self.index[term][0], self.index[term][1]


    def positionalQuery(self, phrase: str) -> Tuple[Dict[str, List[str]], int]:
        """
        Perform a positional query on the positional index.
        :param phrase: The phrase to query.
        :return: The number of documents that contain the phrase and a dictionary of the documents and their positions.
        """

        # Phrase in form ["term1", "term2", ...]
        # Check if all terms are in index and convert them to form [{"term1": (index, freq)}, {"term2": (index, freq)}, ...]
        phrase = phrase.split(" ")
        terms = []

        for p in range(len(phrase)):
            if phrase[p] in stopwords.words("english"):
                continue
            if phrase[p] not in self.index:
                return {}, 0
            else:
                terms.append({phrase[p]: (p, self.index[phrase[p]][0])})

        # Sort in ascending order of frequency
        terms = sorted(terms, key=lambda x: list(x.values())[0][1])

        # Get posting list for term with the lowest frequency and remove it from the list
        result = self.index[list(terms[0].keys())[0]][1]
        term_index = list(terms[0].values())[0][0]
        terms.pop(0)

        while len(terms) > 0:
            # Get posting list and index in phrase for next term
            posting_list = self.index[list(terms[0].keys())[0]][1]
            index = list(terms[0].values())[0][0]
            terms.pop(0)

            # Perform positional query
            for doc in list(result.keys()):
                if doc not in posting_list:
                    result.pop(doc)
                else:
                    # Get positions for term in phrase and term in posting list
                    positions = result[doc]
                    posting_positions = posting_list[doc]

                    # Check if positions are within same distance of each other as in phrase
                    matched_positions = []
                    for p in range(len(positions)):
                        for pp in range(len(posting_positions)):

                            if posting_positions[pp] > positions[p] + abs(index - term_index):
                                break

                            if posting_positions[pp] - positions[p] == index - term_index:
                                matched_positions.append(positions[p])

                    # If no positions match, remove document from result otherwise update positions
                    if len(matched_positions) == 0:
                        result.pop(doc)
                    else:
                        result[doc] = matched_positions

        # Update positions to be relative to phrase
        for doc in result:
            result[doc] = [p - term_index for p in result[doc]]

        return result, len(result)





In [47]:
# Create bigram and positional index
bg = BigramIndex(ORIGINAL_PATH)
psi = PositionalIndex(ORIGINAL_PATH)

# Save indexes
with open("BigramIndex.pickle", "wb") as f:
    pickle.dump(bg, f)

with open("PositionalIndex.pickle", "wb") as f:
    pickle.dump(psi, f)

Finished Building Index


In [48]:
# Load indexes
with open("BigramIndex.pickle", "rb") as f:
    bg = pickle.load(f)

with open("PositionalIndex.pickle", "rb") as f:
    psi = pickle.load(f)

In [49]:
print("--- Phrase Queries ---")
print("Enter the query: ")
print()

n = int(input("Enter the number of queries: "))


for i1 in range(n):
    search_term = input("Enter the search term: ")
    # search_term = "document in number fixed a a words"

    # tokenize, remove punctuation, remove blank strings for both indexes
    search_term = word_tokenize(search_term.lower())
    search_term = [s.translate(str.maketrans("", "", string.punctuation)) for s in search_term]
    search_term = [s for s in search_term if s.strip()]

    # Remove stopwords for bigram index
    search_term2 = [s for s in search_term if s not in stopwords.words("english")]

    query1 = ""
    for j in range(len(search_term2)):
        query1 += search_term2[j] + " "

    query1 = query1.strip()

    # print("Bigram Index")
    # print("Bigram Query", i1 + 1, ": ", query)

    result1, comparisons1 = bg.queryProcess(query1)
    print("Number of documents retrieved for query", i1+1, "using bigram inverted index: ", len(result1))
    print("Name of documents retrieved for query", i1+1, "using bigram inverted index: ", end="")
    for i2 in range(len(result1)):
        print(result1[i2], end="")
        if i2 < len(result1) - 1:
            print(", ", end="")
    print()



    query1 = ""
    for j1 in range(len(search_term)):
        query1 += search_term[j1] + " "

    query1 = query1.strip()

    # print("Positional Index")
    # print("Positional Query", i1 + 1, ": ", query1)

    result2, len_result2 = psi.positionalQuery(query1)
    print("Number of documents retrieved for query", i1+1, "using positional index: ", len_result2)
    print("Name of documents retrieved for query", i1+1, "using positional index: ", end="")

    if result2:
        for doc1 in result2:
            print(doc1, end="")
            if doc1 != list(result2.keys())[-1]:
                print(", ", end="")
    print()

--- Phrase Queries ---
Enter the query: 

Number of documents retrieved for query 1 using bigram inverted index:  8
Name of documents retrieved for query 1 using bigram inverted index: cranfield0007, cranfield0096, cranfield0207, cranfield0344, cranfield0418, cranfield0536, cranfield0610, cranfield1321
Number of documents retrieved for query 1 using positional index:  8
Name of documents retrieved for query 1 using positional index: cranfield0007, cranfield0096, cranfield0207, cranfield0344, cranfield0418, cranfield0536, cranfield0610, cranfield1321
Number of documents retrieved for query 2 using bigram inverted index:  1
Name of documents retrieved for query 2 using bigram inverted index: cranfield0007
Number of documents retrieved for query 2 using positional index:  1
Name of documents retrieved for query 2 using positional index: cranfield0007
